In [18]:
import tensorflow as tf
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential
from keras import layers,regularizers

from skimage import transform
import pickle


In [19]:
def resize_batch(imgs):
    # A function to resize a batch of MNIST images to (32, 32)
    # Args:
    #   imgs: a numpy array of size [batch_size, 28 X 28].
    # Returns:
    #   a numpy array of size [batch_size, 32, 32].
    imgs = imgs.reshape((-1, 28, 28, 1))
    resized_imgs = np.zeros((imgs.shape[0], 32, 32, 1))
    for i in range(imgs.shape[0]):
        resized_imgs[i, ..., 0] = transform.resize(imgs[i, ..., 0], (32, 32))
    return resized_imgs

In [20]:
# please download dataset from git-lfs before running
with open('kannada_semi_1pct.pkl', 'rb') as f:
    kannada_semi = pickle.load(f)
kannada_x_train_labeled = kannada_semi["x_train_labeled"]
kannada_y_train_labeled = kannada_semi["y_train_labeled"]

kannada_x_train_unlabeled = kannada_semi["x_train_unlabeled"]
kannada_y_train_unlabeled = kannada_semi["y_train_unlabeled"]

kannada_x_train = np.concatenate((kannada_x_train_labeled, kannada_x_train_unlabeled), axis=0)
kannada_y_train = np.concatenate((kannada_y_train_labeled, kannada_y_train_unlabeled), axis=0)

kannada_x_val = kannada_semi["x_val"]
kannada_y_val = kannada_semi["y_val"]

kannada_x_test = kannada_semi['x_test']
kannada_y_test = kannada_semi['y_test']

In [21]:
# please download dataset from git-lfs before running
with open('dig_test.pkl', 'rb') as f:
    dig_test = pickle.load(f)
dig_x_test = dig_test['x_test']
dig_y_test = dig_test['y_test']

# Kannada-MNIST Baseline

In [23]:
# Reshaping K-MNIST data from (28,28) to (32,32)

# kannada_x_train_labeled = resize_batch(kannada_x_train_labeled)
# kannada_x_train_unlabeled = resize_batch(kannada_x_train_unlabeled)

kannada_x_train = resize_batch(kannada_x_train)
kannada_x_val = resize_batch(kannada_x_val)
kannada_x_test = resize_batch(kannada_x_test)

In [24]:
# Reshaping Dig-MNIST data from (28,28) to (32,32)
dig_x_test = resize_batch(dig_x_test)

## model and training

In [25]:
bmodel = Sequential()

bmodel.add(layers.Conv2D(32, (4,4), strides=2, padding='same', activation=tf.nn.leaky_relu, input_shape=(32, 32, 1)))
# bmodel.add(layers.LeakyReLU(alpha=0.2))
bmodel.add(layers.Dropout(0.5))

bmodel.add(layers.Conv2D(64, (4,4), strides=2, padding='same', activation=tf.nn.leaky_relu))
# bmodel.add(layers.LeakyReLU(alpha=0.2))
bmodel.add(layers.Dropout(0.5))

bmodel.add(layers.Conv2D(128, (4,4), strides=2, padding='same', activation=tf.nn.leaky_relu))
# bmodel.add(layers.LeakyReLU(alpha=0.2))
bmodel.add(layers.Dropout(0.5))

bmodel.add(layers.Conv2D(256, (4,4), strides=2, padding='same', activation=tf.nn.leaky_relu))
# bmodel.add(layers.LeakyReLU(alpha=0.2))
bmodel.add(layers.Dropout(0.5))

bmodel.add(layers.Conv2D(512, (4,4), strides=2, padding='same', activation=tf.nn.leaky_relu))
# bmodel.add(layers.LeakyReLU(alpha=0.2))
bmodel.add(layers.Dropout(0.5))

bmodel.add(layers.Flatten())
bmodel.add(layers.Dense(10, activation='softmax'))

In [29]:
bmodel.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
bmodel.fit(x=kannada_x_train, y=kannada_y_train, validation_data=(kannada_x_val, kannada_y_val), epochs=10)
# change to 5 epochs for future trainings

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 302s 5ms/step - loss: 0.1670 - accuracy: 0.9481 - val_loss: 0.0566 - val_accuracy: 0.9820
Epoch 2/10
55000/55000 [==============================] - 273s 5ms/step - loss: 0.1216 - accuracy: 0.9637 - val_loss: 0.0413 - val_accuracy: 0.9876
Epoch 3/10
55000/55000 [==============================] - 279s 5ms/step - loss: 0.1188 - accuracy: 0.9661 - val_loss: 0.0392 - val_accuracy: 0.9882
Epoch 4/10
55000/55000 [==============================] - 284s 5ms/step - loss: 0.1138 - accuracy: 0.9668 - val_loss: 0.0299 - val_accuracy: 0.9900
Epoch 5/10
55000/55000 [==============================] - 301s 5ms/step - loss: 0.1102 - accuracy: 0.9681 - val_loss: 0.0417 - val_accuracy: 0.9872
Epoch 6/10
55000/55000 [==============================] - 273s 5ms/step - loss: 0.1145 - accuracy: 0.9673 - val_loss: 0.0450 - val_accuracy: 0.9868
Epoch 7/10
55000/55000 [==============================] - 274s 

## K-MNIST baseline model evaluation

In [30]:
bmodel.evaluate(kannada_x_test, kannada_y_test)

10000/10000 [==============================] - 12s 1ms/step


[0.17482808328717947, 0.9532999992370605]

## Baseline model evaluation on Dig-MNIST dataset

In [31]:
bmodel.evaluate(dig_x_test, dig_y_test)

10240/10240 [==============================] - 13s 1ms/step


[1.8050895781023428, 0.725878894329071]